In [1]:
from vizdoom import *
import os
import numpy as np
import time
import random
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker

2024-08-08 21:28:37.424717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 21:28:37.462311: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 21:28:37.475573: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-08 21:28:37.544647: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-08 21:28:38.519498: W tensorflow/compiler/tf2

In [2]:
game = DoomGame()
game.load_config(os.path.join(scenarios_path, "defend_the_center.cfg"))
game.init()

In [3]:
actions = np.identity(3, dtype=np.uint8)

In [4]:
state = game.get_state()

In [5]:
state.screen_buffer.shape

(3, 240, 320)

In [6]:
state.game_variables

array([ 26., 100.])

In [7]:
episodes = 10 
for episode in range(episodes): 
    # Create a new episode or game 
    game.new_episode()
    # Check the game isn't done 
    while not game.is_episode_finished(): 
        # Get the game state 
        state = game.get_state()
        # Get the game image 
        img = state.screen_buffer
        # Get the game variables - ammo
        info = state.game_variables
        # Take an action
        reward = game.make_action(random.choice(actions),4)
        # Print rewward 
        print('reward:', reward) 
        time.sleep(0.2)
    print('Result:', game.get_total_reward())
    time.sleep(2)

reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 1.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: -1.0
Result: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
rew

In [8]:
game.close()

## introducing the environment

In [9]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import cv2

In [10]:
class VizDoomGym(Env):
    def __init__(self, render=False):
        super().__init__()

        self.game = DoomGame()
        self.game.load_config(os.path.join(scenarios_path, "defend_the_center.cfg"))

        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)

        self.game.init()

        self.observation_space = Box(low=0, high =255, shape=(100,160,1), dtype=np.uint8)
        self.action_space = Discrete(3)

    def step(self, action):
        actions = np.identity(3, dtype=np.uint8)
        reward = self.game.make_action(actions[action], 4)

        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            info = self.game.get_state().game_variables
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0

        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, done, info
    
    def render():
        pass

    def reset(self, seed=None):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        info = self.game.get_state().game_variables
        info = {"info":info}
        return self.grayscale(state), info

    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160, 100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state

    def close(self):
        self.game.close()

In [11]:
env = VizDoomGym(render=True)

In [12]:
state = env.reset()

In [13]:
env.reset()

(array([[[ 5],
         [10],
         [ 7],
         ...,
         [ 4],
         [ 6],
         [10]],
 
        [[ 3],
         [ 3],
         [ 4],
         ...,
         [ 5],
         [ 3],
         [ 7]],
 
        [[11],
         [ 7],
         [10],
         ...,
         [ 9],
         [ 6],
         [ 6]],
 
        ...,
 
        [[75],
         [63],
         [62],
         ...,
         [44],
         [71],
         [60]],
 
        [[15],
         [48],
         [47],
         ...,
         [49],
         [69],
         [47]],
 
        [[22],
         [14],
         [26],
         ...,
         [57],
         [37],
         [39]]], dtype=uint8),
 {'info': array([ 26., 100.])})

In [14]:
env.close()

## training loop callback

In [15]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [16]:
CHECKPOINT_DIR = './train/train_defend_it2'
LOG_DIR = './logs/log_defend_it2'

In [17]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

## training

In [18]:
from stable_baselines3 import PPO

In [19]:
env = VizDoomGym()

In [20]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.0001, n_steps=4096)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [21]:
model.learn(total_timesteps=600000, callback=callback)

Logging to ./logs/log_defend_it2/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 81.7     |
|    ep_rew_mean     | 0.286    |
| time/              |          |
|    fps             | 410      |
|    iterations      | 1        |
|    time_elapsed    | 9        |
|    total_timesteps | 4096     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 84.9         |
|    ep_rew_mean          | 0.458        |
| time/                   |              |
|    fps                  | 307          |
|    iterations           | 2            |
|    time_elapsed         | 26           |
|    total_timesteps      | 8192         |
| train/                  |              |
|    approx_kl            | 0.0055123577 |
|    clip_fraction        | 0.0298       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_vari

KeyboardInterrupt: 

## testing

In [23]:
from stable_baselines3.common.evaluation import evaluate_policy

In [40]:
model = PPO.load('./train/train_defend_it2/best_model_120000')

In [41]:
env = VizDoomGym(render=True)

In [42]:
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10)

In [43]:
mean_reward

13.7

In [44]:
for episode in range(4): 
    obs, _ = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, _, info = env.step(action)
        time.sleep(0.10)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(total_reward, episode))
    time.sleep(1)

Total Reward for episode 13.0 is 0
Total Reward for episode 5.0 is 1
Total Reward for episode 14.0 is 2
Total Reward for episode 15.0 is 3


In [45]:
env.close()